In [15]:
%matplotlib inline
import logging
import Euclid
import matplotlib.pyplot as plt
import astropy.units as u
import numpy as np

from matplotlib import colors, patches, cm
from astropy.table import Table, Column, join
from tqdm.notebook import tqdm
from scipy.stats import gaussian_kde
from scipy.signal import find_peaks
from scikitplot.metrics import plot_confusion_matrix
from IPython.display import set_matplotlib_formats

%elogin
%erun SIM_Tools

from SimSpectra.SimSpectra import SimGalaxySpectra
from ST_DM_MDBTools.Mdb import Mdb

plt.rcParams['figure.figsize'] = (10, 6)
set_matplotlib_formats('pdf')

In [2]:
t = Table.read('/home/aalvarez/Work/Data/SC8/run5/pp_rerun_20210129_with_tu_SNR10.fits')

In [3]:
spec = SimGalaxySpectra(
    sedlib_filename='/home/aalvarez/Work/Data/SC8/TU/Lib/EUC-TEST-GSEDLIB-2015-03-18T171100.000.fits',
    filter_filename='/home/aalvarez/Work/Data/SC8/TU/Lib/EUC-TEST-FILSDSSR01-2015-02-02T203100.000.fits',
    extlib_filename='/home/aalvarez/Work/Data/SC8/TU/Lib/EUC-TEST-GEXTLAWLIB-2015-03-18T203100.000.fits',
    #extmap_filename=[
    #    '/home/aalvarez/Work/Data/SC8/TU/Lib/SFD_dust_1024_ngp.fits',
    #    '/home/aalvarez/Work/Data/SC8/TU/Lib/SFD_dust_1024_sgp.fits',
    #],
    extmap_filename=['/home/aalvarez/Work/Data/SC8/TU/Lib/HFI_CompMap_ThermalDustModel_2048_R1.20_V2.fits'],
    norm_sed='AB.sed',
    wmin=   500E-10,
    wmax=100000E-10,
    nwl=2000
)

INFO    : Use internal AB reference spectrum
2021-02-10T09:25:31CET SimSpectra  INFO : Use internal AB reference spectrum


In [4]:
def get_spectra(obj, sed_template):
    spectra = spec.get_spectra(
        ra=float(obj['RA_MAG']), dec=float(obj['DEC_MAG']), refmag=float(obj['REF_MAG']), absmag=float(obj['REF_MAG_ABS']),
        z=float(obj['Z_OBS']), ised=sed_template, ebv=float(obj['EBV']), extlaw=float(obj['EXT_LAW']), vacuum=True,
        lines={
            'halpha': float(obj['HALPHA_LOGFLAM_EXT_MAG']),
            'hbeta': float(obj['HBETA_LOGFLAM_EXT_MAG']),
            'O2': float(obj['O2_LOGFLAM_EXT_MAG']),
            'O3': float(obj['O3_LOGFLAM_EXT_MAG']),
            'N2': float(obj['N2_LOGFLAM_EXT_MAG']),
            'S2': float(obj['S2_LOGFLAM_EXT_MAG'])
        },
    )
    # Spectra is in Watts
    spectra *= 1e-7
    return np.stack([spec.wgrid.to(u.Angstrom).value, spectra], axis=-1)

Roughly

* From 0 to 8 elliptical
* From 9 to 19 spiral
* From 20 to 30 SB

In [5]:
SPIRAL_LMIN = 9
SB_LMIN = 19.5

t['TRUE_GALCLASS'] = 0
t['TRUE_GALCLASS'][(t['SED_TEMPLATE'] >= SPIRAL_LMIN)] = 1
t['TRUE_GALCLASS'][(t['SED_TEMPLATE'] >= SB_LMIN)] = 2
mask = t['FLAG']==0

In [6]:
sb_bad_0 = t[(t['TRUE_GALCLASS'] == 2) & (t['GALAXY_CLASS'] == 0)]
sb_bad_1 = t[(t['TRUE_GALCLASS'] == 2) & (t['GALAXY_CLASS'] == 1)]
sb_good = t[(t['TRUE_GALCLASS'] == 2) & (t['GALAXY_CLASS'] == 2)]

In [7]:
def hist_mis(attr, ax=None):
    if ax is None:
        ax = plt.gca()
    bins = ax.hist(sb_bad_0[attr], histtype='step', label='Misclassified as elliptical', density=True)
    ax.hist(sb_bad_1[attr], histtype='step', label='Misclassified as spiral', bins=bins[1], density=True)
    ax.hist(sb_good[attr], histtype='step', label='Good', bins=bins[1], density=True)
    ax.set_xlabel(attr)
    ax.legend()

In [8]:
fig, axes = plt.subplots(ncols=2, nrows=3, figsize=(20, 10))
for ax, attr in zip(axes.ravel(), ['SED_TEMPLATE', 'EBV', 'Z_OBS', 'REF_MAG', 'AV', 'EXT_LAW']):
    hist_mis(attr, ax)
plt.tight_layout()

<Figure size 1440x720 with 6 Axes>

In [10]:
sb_bad_0.colnames

['OBJECT_ID',
 'PHZ_PP_MEDIAN_Z',
 'PHZ_PP_MODE_Z',
 'PHZ_PP_68_Z',
 'PHZ_PP_MEDIAN_LOG_STELLARMASS',
 'PHZ_PP_MODE_LOG_STELLARMASS',
 'PHZ_PP_68_LOG_STELLARMASS',
 'PHZ_PP_MEDIAN_SFR',
 'PHZ_PP_MODE_SFR',
 'PHZ_PP_68_SFR',
 'GALAXY_CLASS',
 'PHZ_PP_SFH_TYPE',
 'PHZ_PP_SFH_TAU',
 'FLAG',
 'OBJECT_ID_1',
 'SOURCE_ID',
 'RA_MAG',
 'DEC_MAG',
 'Z_OBS',
 'REF_MAG_ABS',
 'REF_MAG',
 'KAPPA',
 'GAMMA1',
 'GAMMA2',
 'SED_TEMPLATE',
 'EXT_LAW',
 'EBV',
 'HALPHA_LOGFLAM_EXT_MAG',
 'HBETA_LOGFLAM_EXT_MAG',
 'O2_LOGFLAM_EXT_MAG',
 'O3_LOGFLAM_EXT_MAG',
 'N2_LOGFLAM_EXT_MAG',
 'S2_LOGFLAM_EXT_MAG',
 'AV',
 'TRUE_GALCLASS']

In [13]:
nn = Table.read('/home/aalvarez/Work/Data/SC8/run5/pp_rerun_20210129.fits')

In [29]:
tn_bad_0 = join(nn, sb_bad_0, 'OBJECT_ID')

In [30]:
plt.figure()
plt.hist(tn_bad_0['NEIGHBOR_WEIGHTS'].ravel(), bins=2**np.linspace(-15, 0, 10))
plt.xscale('log')
plt.xlabel('Neighbor Weights')
plt.title('SB misclassified as Elliptical')
plt.show()

<Figure size 720x432 with 1 Axes>

In [31]:
tn_good = join(nn, sb_good, 'OBJECT_ID')

In [33]:
plt.figure()
plt.hist(tn_good['NEIGHBOR_WEIGHTS'].ravel(), bins=2**np.linspace(-15, 0, 10))
plt.xscale('log')
plt.xlabel('Neighbor Weights')
plt.title('SB properly classified')
plt.show()

<Figure size 720x432 with 1 Axes>

In [35]:
tn_good['MC_COUNT_GAL_TYPE']

6 .. 74
0 .. 95
5 .. 95
5 .. 95
1 .. 99
40 .. 60
6 .. 94
0 .. 100
0 .. 99
5 .. 95
36 .. 64


In [46]:
fig, axes = plt.subplots(ncols=3)
axes[0].hist(tn_bad_0['MC_COUNT_GAL_TYPE'][:,0])
axes[1].hist(tn_bad_0['MC_COUNT_GAL_TYPE'][:,1])
axes[2].hist(tn_bad_0['MC_COUNT_GAL_TYPE'][:,2])
plt.show()

<Figure size 720x432 with 3 Axes>

In [47]:
fig, axes = plt.subplots(ncols=3)
axes[0].hist(tn_good['MC_COUNT_GAL_TYPE'][:,0])
axes[1].hist(tn_good['MC_COUNT_GAL_TYPE'][:,1])
axes[2].hist(tn_good['MC_COUNT_GAL_TYPE'][:,2])
plt.show()

<Figure size 720x432 with 3 Axes>